# T07 : Machine Learning & Évaluation

## 1. Contexte et Objectifs

Cette étape consiste à entraîner des modèles de Machine Learning supervisés pour prédire la direction future du marché GBP/USD à l'horizon M15. Nous respectons strictement le découpage temporel imposé :
- **Train (2022)** : Apprentissage du modèle.
- **Validation (2023)** : Sélection et hyper-paramétrage (si applicable).
- **Test (2024)** : Évaluation finale (Sanctuarisée).

**Objectif de prédiction** : Classifieur binaire `y`.
- `1` : Hausse (Close[t+1] > Close[t])
- `0` : Baisse ou Stagnation (Close[t+1] <= Close[t])

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import os

# Configuration Graphique
sns.set_theme(style="whitegrid")
plt.rcParams.update({
    "figure.facecolor": "#FAF0E6",
    "axes.facecolor": "#F5F5DC",
    "grid.color": "#E0D0C0",
    "text.color": "#5D4037",
    "axes.prop_cycle": plt.cycler(color=['#8D6E63', '#A1887F', '#D7CCC8'])
})

DATA_DIR = "data/features"
MODELS_DIR = "models/v1"
os.makedirs(MODELS_DIR, exist_ok=True)

FILES = {
    "TRAIN": "GBPUSD_M15_2022_features.csv",
    "VAL": "GBPUSD_M15_2023_features.csv",
    "TEST": "GBPUSD_M15_2024_features.csv"
}

## 2. Préparation des Données (Data Loading & Preprocessing)

Nous devons :
1. Charger les features.
2. Créer la cible (`target`) : Le signe du rendement futur.
3. Séparer X (features) et y (target).
4. Normaliser les données en apprenant les paramètres (`mean`, `std`) **uniquement sur le Train**.

In [ ]:
def load_and_prep(filename):
    path = os.path.join(DATA_DIR, filename)
    if not os.path.exists(path):
        raise FileNotFoundError(f"Fichier introuvable: {path}")
    
    df = pd.read_csv(path, parse_dates=['timestamp'], index_col='timestamp')
    
    # Création de la target (futur immédiat)
    # Shift(-1) permet de regarder la bougie SUIVANTE
    # Attention : la dernière ligne aura un NaN et devra être supprimée
    df['target_return'] = df['close_15m'].shift(-1) - df['close_15m']
    df['target'] = (df['target_return'] > 0).astype(int)
    
    # Suppression de la dernière ligne (pas de futur connu)
    df.dropna(inplace=True)
    
    return df

# Chargement
df_train = load_and_prep(FILES["TRAIN"])
df_val = load_and_prep(FILES["VAL"])
df_test = load_and_prep(FILES["TEST"])

print(f"Train size : {df_train.shape}")
print(f"Val size   : {df_val.shape}")
print(f"Test size  : {df_test.shape}")

# Sélection des features (tout sauf les colonnes 'target' et les prix bruts si nécessaire)
# On exclut les colonnes 'futur' ou 'target'
drop_cols = ['target', 'target_return', 'open_15m', 'high_15m', 'low_15m', 'close_15m', 'volume_15m', 'tick_count']
# On garde les indicateurs calculés
features_cols = [c for c in df_train.columns if c not in drop_cols]

X_train = df_train[features_cols]
y_train = df_train['target']

X_val = df_val[features_cols]
y_val = df_val['target']

X_test = df_test[features_cols]
y_test = df_test['target']

# Normalisation (StandardScaler)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
# Important : On utilise le scaler fitté sur le TRAIN pour transformer VAL et TEST
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f"\nFeatures ({len(features_cols)}) : {features_cols}")

## 3. Modélisation : Baseline & Random Forest

Nous commençons par une régression logistique simple comme 'baseline', puis un Random Forest.

In [ ]:
models = {
    "Logistic Regression": LogisticRegression(random_state=42, class_weight='balanced'),
    "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42, class_weight='balanced') # Max depth limité pour éviter overfitting
}

results = {}

for name, model in models.items():
    print(f"\n--- Entraînement {name} ---")
    model.fit(X_train_scaled, y_train)
    
    # Prédictions
    train_pred = model.predict(X_train_scaled)
    val_pred = model.predict(X_val_scaled)
    
    # Stockage des modèles et stats
    acc_train = accuracy_score(y_train, train_pred)
    acc_val = accuracy_score(y_val, val_pred)
    
    print(f"Accuracy Train : {acc_train:.4f}")
    print(f"Accuracy Val   : {acc_val:.4f}")
    print("Rapport de classification (Val) :")
    print(classification_report(y_val, val_pred))
    
    results[name] = model

## 4. Évaluation Financière (Backtest Vectorisé)

L'accuracy ne suffit pas. Nous devons vérifier si le modèle génère du profit.
Simulation simple : 
- Si Pred = 1 (Hausse) -> Achat (Long)
- Si Pred = 0 (Baisse) -> Vente (Short) ou Cash (Flat)? 
Pour simplifier ici, supposons une stratégie 'Long Only' filtrée (on n'achète que si Pred=1) ou 'Long/Short'.
Prenons le cas **Long/Short** pour maximiser l'impact de la prédiction : 
- Signal = 1 -> Return du marché
- Signal = 0 -> - Return du marché (Short)

*Note : Sans coûts de transaction pour l'instant (brut).*

In [ ]:
def backtest_model(model, X, df_original, title=""):
    preds = model.predict(X)
    
    # Stratégie : 1 -> Long (+return), 0 -> Short (-return)
    # Mapping 0 -> -1
    signals = np.where(preds == 1, 1, -1)
    
    # Calcul du PnL cumulé
    # On multiplie le signal par le retour futur 'target_return'
    strategy_returns = signals * df_original['target_return']
    cumulative_returns = strategy_returns.cumsum()
    
    # Baseline Buy & Hold
    market_returns = df_original['target_return'].cumsum()
    
    # Plot
    plt.figure(figsize=(12, 6))
    plt.plot(df_original.index, cumulative_returns, label='Modèle Strategy', color='#8D6E63')
    plt.plot(df_original.index, market_returns, label='Buy & Hold', color='gray', alpha=0.5, linestyle='--')
    plt.title(f"PnL Cumulé - {title}", fontweight='bold')
    plt.ylabel("Pips / Points cumulés")
    plt.legend()
    plt.show()
    
    return cumulative_returns.iloc[-1]

# Évaluation sur le Test Set (2024)
print("\n=== ÉVALUATION FINALE SUR TEST (2024) ===")
best_model = results['Random Forest'] # On choisit le RF par défaut

final_pnl = backtest_model(best_model, X_test_scaled, df_test, title="Test 2024 (Random Forest)")
print(f"Profit Final (Points) sur 2024 : {final_pnl:.5f}")

## 5. Analyse de l'Importance des Features

Quelles variables ont le plus influencé le modèle de Random Forest ?

In [ ]:
importances = best_model.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(12, 6))
plt.title("Importance des Features (Random Forest)")
plt.bar(range(X_train.shape[1]), importances[indices], align="center", color='#A1887F')
plt.xticks(range(X_train.shape[1]), [features_cols[i] for i in indices], rotation=45, ha='right')
plt.xlim([-1, X_train.shape[1]])
plt.tight_layout()
plt.show()

## 6. Conclusion T07

Ce notebook a permis d'entraîner et valider une première approche ML.

**Points Clés :**
- Le split temporel a été respecté.
- La normalisation est ancrée sur le Train set.
- Une évaluation financière brute a été réalisée sur 2024.

**Limitations :**
- Le modèle reste basique (Random Forest standard).
- Les coûts de transaction (spread) ne sont pas inclus dans le backtest, ce qui rend les résultats probablement optimistes.
- L'horizon de prédiction est très court (M15 suivant).

**Pour la suite (T08 - RL) :** Le Reinforcement Learning pourra potentiellement mieux gérer la séquence de décision (Garder vs Vendre) et intégrer les coûts de manière native dans la fonction de récompense.